In [11]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")

# Define compliance violation categories
insider_trading_terms = [
    "buy before announcement",
    "sell before news",
    "confidential merger information",
    "unreleased earnings",
    "material nonpublic information",
    "MNPI",
    "tip from insider",
    "quiet period violation",
    "trading window closed",
    "blackout period"
]

market_manipulation_terms = [
    "pump and dump",
    "coordinated buying",
    "artificial price inflation",
    "wash trading",
    "spoofing",
    "layering orders",
    "painting the tape",
    "front running"
]

money_laundering_terms = [
    "structuring deposits",
    "smurfing",
    "shell company transfer",
    "offshore account",
    "cash intensive business",
    "high risk jurisdiction",
    "beneficial owner concealment",
    "unusual wire transfer pattern"
]

gdpr_violation_terms = [
    "personal data sharing",
    "customer information disclosed",
    "privacy breach",
    "consent not obtained",
    "data protection violation",
    "right to be forgotten ignored",
    "cross-border data transfer"
]

In [12]:
# Create PhraseMatcher
compliance_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

# Add patterns
compliance_matcher.add("INSIDER_TRADING", [nlp.make_doc(term) for term in insider_trading_terms])
compliance_matcher.add("MARKET_MANIPULATION", [nlp.make_doc(term) for term in market_manipulation_terms])
compliance_matcher.add("MONEY_LAUNDERING", [nlp.make_doc(term) for term in money_laundering_terms])
compliance_matcher.add("GDPR_VIOLATION", [nlp.make_doc(term) for term in gdpr_violation_terms])

# Sample email monitoring
email_text = """
From: trader@globalfinance.com
To: colleague@globalfinance.com
Subject: Quick heads up

Hey - just got confidential merger information from our client. 
The deal closes next week. You might want to buy before announcement.
Keep this quiet, we're in the blackout period.
"""

doc = nlp(email_text)
matches = compliance_matcher(doc)

print("🚨 COMPLIANCE ALERTS DETECTED:")
print("=" * 70)
for match_id, start, end in matches:
    violation_type = nlp.vocab.strings[match_id]
    matched_text = doc[start:end].text
    print(f"\n⚠️  Violation Type: {violation_type}")
    print(f"   Matched Text: '{matched_text}'")
    print(f"   Risk Level: HIGH")

🚨 COMPLIANCE ALERTS DETECTED:

⚠️  Violation Type: INSIDER_TRADING
   Matched Text: 'confidential merger information'
   Risk Level: HIGH

⚠️  Violation Type: INSIDER_TRADING
   Matched Text: 'buy before announcement'
   Risk Level: HIGH

⚠️  Violation Type: INSIDER_TRADING
   Matched Text: 'blackout period'
   Risk Level: HIGH


In [13]:
from spacy.pipeline import EntityRuler

ruler = nlp.add_pipe("entity_ruler", before="ner")

patterns = [
    # Stock ticker patterns
    {
        "label": "STOCK_TICKER",
        "pattern": [
            {"TEXT": {"REGEX": "^[A-Z]{1,5}$"}},
            {"LOWER": "stock"}
        ]
    },
    {
        "label": "STOCK_TICKER",
        "pattern": [
            {"TEXT": {"REGEX": "^[A-Z]{1,5}$"}},
            {"LOWER": {"IN": ["shares", "equity"]}}
        ]
    },

    # ISIN codes
    {
        "label": "ISIN_CODE",
        "pattern": [
            {"TEXT": {"REGEX": "^[A-Z]{2}[A-Z0-9]{9}[0-9]$"}}
        ]
    },

    # Account numbers
    {
        "label": "ACCOUNT_NUMBER",
        "pattern": [
            {"TEXT": {"REGEX": "^ACC-\\d{8}$"}}
        ]
    },
    {
        "label": "ACCOUNT_NUMBER",
        "pattern": [
            {"LOWER": "account"},
            {"TEXT": {"REGEX": "^\\d{10,12}$"}}
        ]
    },

    # Transaction IDs
    {
        "label": "TRANSACTION_ID",
        "pattern": [
            {"TEXT": {"REGEX": "^TXN-[A-Z0-9]{12}$"}}
        ]
    },

    # Large currency amounts
    {
        "label": "LARGE_AMOUNT",
        "pattern": [
            {"TEXT": "$"},
            {"TEXT": {"REGEX": "^[1-9]\\d{6,}$"}}
        ]
    },

    # Compliance terms
    {"label": "COMPLIANCE_TERM", "pattern": "material nonpublic information"},
    {"label": "COMPLIANCE_TERM", "pattern": "Chinese Wall"},
    {"label": "COMPLIANCE_TERM", "pattern": "Information Barrier"},
    {"label": "COMPLIANCE_TERM", "pattern": [{"LOWER": "mnpi"}]},
    {"label": "COMPLIANCE_TERM", "pattern": [{"LOWER": "aml"}]},
    {"label": "COMPLIANCE_TERM", "pattern": [{"LOWER": "kyc"}]},

    # Suspicious actions
    {
        "label": "SUSPICIOUS_ACTION",
        "pattern": [{"LOWER": "urgent"}, {"LOWER": "wire"}, {"LOWER": "transfer"}]
    },
    {
        "label": "SUSPICIOUS_ACTION",
        "pattern": [{"LOWER": "delete"}, {"LOWER": "all"}, {"LOWER": "emails"}]
    },
    {
        "label": "SUSPICIOUS_ACTION",
        "pattern": [{"LOWER": "destroy"}, {"LOWER": "documents"}]
    },
    {
        "label": "SUSPICIOUS_ACTION",
        "pattern": [{"LOWER": "cash"}, {"LOWER": "transaction"}]
    },
]

ruler.add_patterns(patterns)


In [14]:
# Monitor suspicious communication
communication = """
URGENT: Need to execute wire transfer of $15000000 to account ACC-98765432 
in Cayman Islands today. ISIN US0378331005 position must be liquidated.
Use TXN-ABC123XYZ789. Delete all emails after completion.
This involves MNPI from the board meeting. Chinese Wall protocols ignored.
"""

doc = nlp(communication)

print("\n💼 FINANCIAL ENTITIES EXTRACTED:")
print("=" * 70)
for ent in doc.ents:
    print(f"{ent.label_:25} | {ent.text}")

# Calculate risk score
risk_score = 0
risk_factors = []

for ent in doc.ents:
    if ent.label_ == "SUSPICIOUS_ACTION":
        risk_score += 30
        risk_factors.append(f"Suspicious action: {ent.text}")
    elif ent.label_ == "COMPLIANCE_TERM":
        risk_score += 25
        risk_factors.append(f"Compliance term: {ent.text}")
    elif ent.label_ == "LARGE_AMOUNT":
        risk_score += 20
        risk_factors.append(f"Large amount: {ent.text}")

print(f"\n🎯 RISK ASSESSMENT:")
print("=" * 70)
print(f"Risk Score: {risk_score}/100")
print(f"Risk Level: {'CRITICAL' if risk_score > 60 else 'HIGH' if risk_score > 30 else 'MEDIUM'}")
print(f"\nRisk Factors:")
for factor in risk_factors:
    print(f"  • {factor}")


💼 FINANCIAL ENTITIES EXTRACTED:
LARGE_AMOUNT              | $15000000
ACCOUNT_NUMBER            | ACC-98765432
LOC                       | Cayman Islands
DATE                      | today
ORG                       | ISIN
ISIN_CODE                 | US0378331005
ORG                       | TXN
ISIN_CODE                 | ABC123XYZ789
SUSPICIOUS_ACTION         | Delete all emails
COMPLIANCE_TERM           | MNPI
COMPLIANCE_TERM           | Chinese Wall

🎯 RISK ASSESSMENT:
Risk Score: 100/100
Risk Level: CRITICAL

Risk Factors:
  • Large amount: $15000000
  • Suspicious action: Delete all emails
  • Compliance term: MNPI
  • Compliance term: Chinese Wall


In [15]:
from spacy.matcher import Matcher

context_matcher = Matcher(nlp.vocab)

# Detect temporal patterns (urgency + financial action)
urgency_patterns = [
    [{"LOWER": {"IN": ["urgent", "immediately", "asap", "rush"]}}, 
     {"OP": "*", "IS_SPACE": False}, 
     {"LOWER": {"IN": ["buy", "sell", "trade", "execute", "transfer"]}}],
    
    # Before/after patterns (timing suspicious)
    [{"LOWER": {"IN": ["before", "prior"]}}, 
     {"LOWER": {"IN": ["announcement", "earnings", "news", "release", "disclosure"]}}],
     
    # Secrecy patterns
    [{"LOWER": {"IN": ["confidential", "secret", "private", "don't"]}},
     {"OP": "?"},
     {"LOWER": {"IN": ["tell", "share", "disclose", "mention", "discuss"]}}],
]

for pattern in urgency_patterns:
    context_matcher.add("HIGH_RISK_CONTEXT", [pattern])

# Test
test_message = "URGENT: Buy 50,000 AAPL shares immediately before tomorrow's earnings announcement. Don't tell compliance."

doc = nlp(test_message)
matches = context_matcher(doc)

print("\n⚡ HIGH-RISK CONTEXT DETECTED:")
print("=" * 70)
for match_id, start, end in matches:
    print(f"Pattern: {doc[start:end].text}")
    print(f"Context: Suspicious timing/secrecy detected")


⚡ HIGH-RISK CONTEXT DETECTED:
Pattern: URGENT: Buy
Context: Suspicious timing/secrecy detected


In [23]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.pipeline import EntityRuler


class ComplianceMonitor:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.setup_matchers()

    def setup_matchers(self):
        # -------- Phrase Matcher (HIGH risk) --------
        self.compliance_matcher = PhraseMatcher(
            self.nlp.vocab,
            attr="LOWER"
        )

        high_risk_phrases = [
            "material nonpublic information",
            "before announcement",
            "insider information",
            "guaranteed profit"
        ]

        patterns = [self.nlp.make_doc(text) for text in high_risk_phrases]
        self.compliance_matcher.add("HIGH_RISK_PHRASE", patterns)

        # -------- Entity Ruler (MEDIUM risk) --------
        ruler = self.nlp.add_pipe("entity_ruler", before="ner")

        entity_patterns = [
            {"label": "COMPLIANCE_TERM", "pattern": [{"LOWER": "mnpi"}]},
            {"label": "SUSPICIOUS_ACTION", "pattern": [{"LOWER": "before"}, {"LOWER": "announcement"}]},
            {"label": "SUSPICIOUS_ACTION", "pattern": [{"LOWER": "buy"}, {"TEXT": {"REGEX": "^[A-Z]{1,5}$"}}, {"LOWER": "stock"}]},
        ]

        ruler.add_patterns(entity_patterns)

    def analyze_message(self, message_text, sender, recipient, timestamp):
        doc = self.nlp(message_text)

        alerts = []
        risk_score = 0

        # -------- Phrase violations --------
        phrase_matches = self.compliance_matcher(doc)
        for match_id, start, end in phrase_matches:
            alerts.append({
                "type": self.nlp.vocab.strings[match_id],
                "text": doc[start:end].text,
                "severity": "HIGH",
                "sender": sender,
                "recipient": recipient,
                "timestamp": timestamp
            })
            risk_score += 40

        # -------- Entity violations --------
        for ent in doc.ents:
            if ent.label_ in ["SUSPICIOUS_ACTION", "COMPLIANCE_TERM"]:
                alerts.append({
                    "type": ent.label_,
                    "text": ent.text,
                    "severity": "MEDIUM",
                    "timestamp": timestamp
                })
                risk_score += 25

        if risk_score >= 50:
            return {
                "alert_id": f"ALERT-{timestamp}",
                "risk_score": risk_score,
                "severity": self._calculate_severity(risk_score),
                "violations": alerts,
                "requires_review": True,
                "escalate_to": "Compliance Officer" if risk_score > 80 else "Supervisor"
            }

        return None

    def _calculate_severity(self, score):
        if score > 80:
            return "CRITICAL"
        elif score > 50:
            return "HIGH"
        elif score > 30:
            return "MEDIUM"
        return "LOW"


In [24]:
monitor = ComplianceMonitor()

# Real-time monitoring
message = "Just heard about the merger with TechCorp. Buy TECH stock before announcement tomorrow."
alert = monitor.analyze_message(
    message_text=message,
    sender="trader1@bank.com",
    recipient="trader2@bank.com",
    timestamp="2024-02-15 09:45:00"
)

if alert:
    print(f"\n🔴 COMPLIANCE ALERT GENERATED:")
    print(f"Alert ID: {alert['alert_id']}")
    print(f"Risk Score: {alert['risk_score']}")
    print(f"Severity: {alert['severity']}")
    print(f"Escalate to: {alert['escalate_to']}")
    print(f"\nViolations Detected: {len(alert['violations'])}")


🔴 COMPLIANCE ALERT GENERATED:
Alert ID: ALERT-2024-02-15 09:45:00
Risk Score: 90
Severity: CRITICAL
Escalate to: Compliance Officer

Violations Detected: 3
